In [1]:
#import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import math
import statsmodels.api as sm

In [2]:
df = pd.read_csv(r"C:\Users\thema\OneDrive\Projects\__CTP4900__\Data\world_Bank.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\thema\\OneDrive\\Projects\\__CTP4900__\\Data\\world_Bank.csv'

In [ ]:
df.head(10)

## Modify the dataframe to better fit the model

In [ ]:
df_melt = pd.melt(df, id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'], 
                  var_name='Year', value_name='Value')

In [ ]:
df_melt.head()

In [ ]:
df_melt['Year'] = df_melt['Year'].str.extract('(\d{4})')[0]

In [ ]:
df_restructured = df_melt.pivot_table(index=['Country Name', 'Country Code', 'Year'], 
                                      columns='Series Name', 
                                      values='Value', 
                                      aggfunc='first').reset_index()

In [ ]:
df_restructured.head()

In [ ]:
# replace column names to have '_' over ' '
df_restructured.columns.name = None  # Remove the name for columns
df_restructured.rename(columns=lambda x: x.replace(', ', '_').replace(' ', '_').lower(), inplace=True)

In [ ]:
df_restructured = df_restructured.replace('..', np.nan)

In [ ]:
columns_to_exclude = ['country_name', 'country_code', 'Series_Name']
for column in df_restructured.columns:    
    if column not in columns_to_exclude and df_restructured[column].dtype == 'object':
        df_restructured[column] = df_restructured[column].astype(float)

In [ ]:
pd.to_csv()

## Modelling phase

### 2nd iteration

In [ ]:
remove_cols = ['births_attended_by_skilled_health_staff_(%_of_total)',
 'co2_emissions_(metric_tons_per_capita)',
 'electric_power_consumption_(kwh_per_capita)',
 'energy_use_(kg_of_oil_equivalent_per_capita)',
 'immunization_measles_(%_of_children_ages_12-23_months)',
 'mobile_cellular_subscriptions_(per_100_people)',
 'personal_remittances_paid_(current_us$)',
 'agriculture_forestry_and_fishing_value_added_(%_of_gdp)',
 'exports_of_goods_and_services_(%_of_gdp)',
 'gdp_(current_us$)',
 'gdp_growth_(annual_%)',
 'gdp_per_capita_(current_us$)',
 'gross_capital_formation_(%_of_gdp)',
 'imports_of_goods_and_services_(%_of_gdp)',
 'industry_(including_construction)_value_added_(%_of_gdp)',
 'inflation_gdp_deflator_(annual_%)',
 'market_capitalization_of_listed_domestic_companies_(%_of_gdp)',
 'merchandise_trade_(%_of_gdp)',
 'military_expenditure_(%_of_gdp)',
 'revenue_excluding_grants_(%_of_gdp)',
 'water_productivity_total_(constant_2015_us$_gdp_per_cubic_meter_of_total_freshwater_withdrawal)',
 'births_attended_by_skilled_health_staff_(%_of_total)',
 'contraceptive_prevalence_any_method_(%_of_married_women_ages_15-49)',
 'electric_power_consumption_(kwh_per_capita)',
 'energy_use_(kg_of_oil_equivalent_per_capita)',
 'external_debt_stocks_total_(dod_current_us$)',
 'high-technology_exports_(%_of_manufactured_exports)',
 'income_share_held_by_lowest_20%',
 'market_capitalization_of_listed_domestic_companies_(%_of_gdp)',
 'poverty_headcount_ratio_at_national_poverty_lines_(%_of_population)',
 'prevalence_of_underweight_weight_for_age_(%_of_children_under_5)',
 'revenue_excluding_grants_(%_of_gdp)',
 'total_debt_service_(%_of_gni)']

In [ ]:
good_cols = [col for col in df_restructured.columns if col not in remove_cols]
good_cols.remove('country_name')
good_cols.remove('country_code')
good_cols

In [ ]:
df_non_object.columns

In [ ]:
scaler = StandardScaler()

In [ ]:
f = open('summary.txt', 'w')
good_features = []
df_non_object = df_restructured.select_dtypes(include=[ 'number' ])
for col in df_non_object.columns:
    df_v2 = df_restructured[[col, 'gdp_(current_us$)']].dropna()
    
    scaled_array = scaler.fit_transform(df_v2)
    df_v2 = pd.DataFrame(scaled_array, columns=df_v2.columns)
    
    X = df_v2[col]
    y = df_v2['gdp_(current_us$)']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)
    
    lin_reg = sm.OLS(y_train, X_train).fit()

    # Predict on the testing set
    y_pred = lin_reg.predict(X_test)

    # Calculate MSE using the testing set
    mse = ((y_test - y_pred) ** 2).mean()

    print(f"Test RMSE: {math.sqrt(mse)}, mean_val: {y_pred.mean()}")
    print(lin_reg.summary())
    f.write(str(lin_reg.summary()))
    adjusted_r_squared = lin_reg.rsquared_adj
    if adjusted_r_squared > 0.1:
        good_features.append(col)

In [ ]:
y_pred

In [ ]:
good_features

In [ ]:
df_v2 = df_restructured[['foreign_direct_investment_net_inflows_(bop_current_us$)',
 'external_debt_stocks_total_(dod_current_us$)',
 'forest_area_(sq._km)',
 'net_migration',
 'population_total',
 'surface_area_(sq._km)',
 'gdp_(current_us$)']].dropna()
    
scaled_array = scaler.fit_transform(df_v2)
df_v2 = pd.DataFrame(scaled_array, columns=df_v2.columns)

X = df_v2.drop(columns = 'gdp_(current_us$)')
y = df_v2['gdp_(current_us$)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

lin_reg = sm.OLS(y_train, X_train).fit()

# Predict on the testing set
y_pred = lin_reg.predict(X_test)

# Calculate MSE using the testing set
mse = ((y_test - y_pred) ** 2).mean()

print(f"Test RMSE: {math.sqrt(mse)}, mean_val: {y_pred.mean()}")
print(lin_reg.summary())